### Note: Set the destinations with images appropriately.

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
from skimage import io
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_gray_feature_vec(r,c,i):
    row_ind_norm = (r-0)/(R-1)
    col_ind_norm = (c-0)/(C-1)
    gray_lin_norm = (i-min_i)/(max_i-min_i)
    return [row_ind_norm, col_ind_norm, gray_lin_norm]

def get_color_feature_vec(r,c,i,red,g,b):
    row_ind_norm = (r-0)/(R-1)
    col_ind_norm = (c-0)/(C-1)
    red_lin_norm = (i[0]-red[0])/(red[1]-red[0])
    green_lin_norm = (i[1]-g[0])/(g[1]-g[0])
    blue_lin_norm = (i[2]-b[0])/(b[1]-b[0])
    return [row_ind_norm, col_ind_norm, red_lin_norm, green_lin_norm, blue_lin_norm ]

def colors(n):
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    step = 256 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append([r,g,b]) 
    return ret

def perform_kmeans(num_clusters, gray_data, image, R, C):
    kmeans = KMeans(n_clusters=num_clusters, random_state=10)
    kmeans.fit(gray_data)
    labels = kmeans.predict(gray_data)
    get_clustered_image(labels, num_clusters, image, R, C)

def get_clustered_image(predicted_labels, n_labels, orig_img, R, C):
    color_dict = dict()
    all_colors = colors(n_labels)
    for i in range(n_labels):
        color_dict[i] = all_colors[i]
    compressed = []
    for label in predicted_labels:
        compressed.append(color_dict[label])
    clust_image = np.array(compressed).ravel().reshape(R,C,3)
    io.imsave("{}_{}.png".format(orig_img, n_labels), clust_image)
    print("Image saved.")

In [3]:
def parse_gray_img(image):
    img = io.imread("{}.png".format(image), as_grey=True)    
    R = img.shape[0]
    C = img.shape[1]
    min_i = min(img.ravel())
    max_i = max(img.ravel())
    
    data = pd.DataFrame({"row_ind_norm": [], "col_ind_norm": [], "gray_lin_norm": []})
    
    try:
        data = pickle.load(open('data_ob_{}'.format(image),'rb'))
    except:
        i = 0
        for r, row in enumerate(img):
            for c, item in enumerate(row):
                data.loc[i] = get_gray_feature_vec(r,c,item)
                i += 1
        pickle.dump(data, open('data_ob_{}'.format(image),'wb'))
    
    for cluster in [2,3,4]:
        perform_kmeans(cluster, data, image, R, C)

In [4]:
parse_gray_img("image1")

Image saved.
Image saved.
Image saved.


In [5]:
parse_gray_img("image2")

Image saved.
Image saved.
Image saved.


In [6]:
def parse_color_img(image):
    img = io.imread("{}.png".format(image))    
    R = img.shape[0]
    C = img.shape[1]
    min_red = min(img[:,:,0].ravel())
    max_red = max(img[:,:,0].ravel())
    min_green = min(img[:,:,1].ravel())
    max_green = max(img[:,:,1].ravel())
    min_blue = min(img[:,:,2].ravel())
    max_blue = max(img[:,:,2].ravel())
    data = pd.DataFrame({"row_ind_norm": [], "col_ind_norm": [], "red_lin_norm": [], "green_lin_norm": [], "blue_lin_norm": []})
    
    try:
        data = pickle.load(open('data_ob_{}'.format(image),'rb'))
    except:
        i = 0
        for r, row in enumerate(img):
            for c, item in enumerate(row):
                data.loc[i] = get_color_feature_vec(r,c,item,(min_red, max_red), (min_green, max_green), (min_blue, max_blue))
                i += 1
        pickle.dump(data, open('data_ob_{}'.format(image),'wb'))
    
    for cluster in [10]:
        perform_kmeans(cluster, data, image, R, C)

In [7]:
parse_color_img("image1")

Image saved.


In [8]:
parse_color_img("image4")

Image saved.
